# 14-day cumulative number of COVID-19 cases per 100 000 for districts (Landkreise) in Germany

At the end of the page, we provide a detailed description of how the numbers are calculated.

### Compute data

In [ ]:
import datetime as dt
import pandas as pd
pd.set_option("max_rows", None)
from oscovida import fetch_data_germany, \
    germany_get_population, get_country_data

# get districts
germany = fetch_data_germany()
districts = sorted(germany['Landkreis'].drop_duplicates())

In [ ]:
data = []
yesterday = dt.date.today() - dt.timedelta(days=1) 

for i, district in enumerate(districts):
    if i % 100 == 0:
        print(f"Processing {i}/412 ({district})")
    c, _, _ = get_country_data(country="Germany", 
                               subregion=district) 
    if c.index[-1].date() < yesterday:
        print(f"{district}: last data is from {c.index[-1].date()}")
        origin = c.index[0].date()
        # Fill data series forward up to yesterday
        new_idx = pd.date_range(origin, periods=(yesterday - origin).days, freq='D')
        c.reindex(new_idx, method='pad')
    c = c[-15:]

    population = germany_get_population(landkreis=district)
    new_cases = int(c[-1] - c[-15]) 
    incidence = new_cases / population * 100000. 
    data += [(district, population, new_cases, round(incidence, 1))]


In [ ]:
# sort, and ignore SK and LK for sorting
data.sort(key=lambda x: x[0].replace("SK ", "").replace("LK ", ""))

# turn into pandas DataFrame for easier display
table = pd.DataFrame(data, 
                     columns=["district", "population", 
                              "new cases", "14-day-incidence"]
                    ).set_index("district")

# Show last update date
import time
print(f"Last updated {time.asctime()}")

## Table for all districts (=Landkreise)

In [ ]:
table

## Table sorted by 14-day-incidence

In [ ]:
table.sort_values(by="14-day-incidence", ascending=False)

## Tutorial: Detailed calculation for one district

In [ ]:
cases, deaths, label = get_country_data("Germany", subregion="SK Hamburg")

Cumulative cases yesterday (numbers for today are not known yet):

In [ ]:
c_y = cases[-1]
c_y

Cumulative cases 15 days ago (i.e. 14 days before yesterday)

In [ ]:
c_15 = cases[-15]
c_15

New cases from the last 14 days

In [ ]:
c_new = c_y - c_15 
c_new

Get the population data

In [ ]:
population = germany_get_population(landkreis="SK Hamburg")
population

Compute the 14-day incidence per 100000, i.e. the cumulative number of new infections in the last 14 days, normalised by the country's population in units of 100000:

In [ ]:
incidence = c_new / (population/100000)
incidence

---------------

In [ ]:
import oscovida
oscovida.display_binder_link('14-day-incidence-germany.ipynb')